# Sample Code Demo

In [1]:
import torch
from torch import nn, Tensor
from torch.utils.data import random_split, DataLoader
import gc

from datautils import get_fixed_frame_data, FixedFramesDataset
from trainutils import device, train


## Hyper Parameters

In [2]:
nframes = 2
batch_size = 512
trial_name = "demo"

## Define Dataset

In [3]:
train_X, train_y = get_fixed_frame_data(nframes, feat_dir="./data/libriphone/feat/train",
                                        split_filepath="./data/libriphone/train_split.txt",
                                        labels_filepath="./data/libriphone/train_labels.txt")
train_dataset = FixedFramesDataset(train_X, train_y)
del train_X, train_y
gc.collect()

train_len = int(len(train_dataset) * 0.8)
valid_len = len(train_dataset) - train_len
train_dataset, valid_dataset = random_split(train_dataset, [train_len, valid_len])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


  0%|          | 0/4286 [00:00<?, ?it/s]

unsupported pickle protocol: 10
[WARNING]: ./data/libriphone/feat/train/481-123719-0012.pt.pt failed to be loaded.
'utf-8' codec can't decode byte 0x81 in position 36: invalid start byte
[WARNING]: ./data/libriphone/feat/train/1183-128659-0014.pt.pt failed to be loaded.


## Define Model

In [4]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim: int, num_classes: int = 41) -> None:
        super(MLPClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, num_classes)
        )

    def forward(self, X: Tensor) -> Tensor:
        return self.fc(X)


In [5]:
model = MLPClassifier(input_dim=5 * 39, num_classes=41).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [6]:
train(train_dataloader, valid_dataloader, model, criterion, optimizer, 200, 10, trial_name)

2022-11-30 09:28:35.081345: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


  0%|          | 0/200 [00:00<?, ?it/s]

Saving model with acc 0.180
Saving model with acc 0.288
Saving model with acc 0.331
Saving model with acc 0.354
Saving model with acc 0.368
Saving model with acc 0.377
Saving model with acc 0.385
Saving model with acc 0.391
Saving model with acc 0.398
Saving model with acc 0.403
Saving model with acc 0.407
Saving model with acc 0.411
Saving model with acc 0.413
Saving model with acc 0.416
Saving model with acc 0.417
Saving model with acc 0.419
Saving model with acc 0.420
Saving model with acc 0.423
Saving model with acc 0.425
Saving model with acc 0.426


## Prediction

In [12]:
test_X = get_fixed_frame_data(nframes, feat_dir="./data/libriphone/feat/test",
                                        split_filepath="./data/libriphone/test_split.txt")
test_dataset = FixedFramesDataset(test_X)
del test_X
gc.collect()

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

from trainutils import prediction

model.load_state_dict(torch.load(f"././models/{trial_name}.ckpt"))
prediction(test_dataloader, model)


  0%|          | 0/1078 [00:00<?, ?it/s]

  0%|          | 0/1263 [00:00<?, ?it/s]